Jeg prøver en konkurrence på kaggle, hvor jeg skal bruge unsupervised k-means clustering.

https://www.kaggle.com/competitions/k-means-clustering-for-heart-disease-analysis/overview

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

split data op i trænings og test data

In [36]:
data = pd.read_csv('data/heart_disease_filtered.csv')
X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)

print(X_train['id'].count())
print(X_test['id'].count())

239
60
